In [3]:
import cv2
import numpy as np


min_contour_width = 40      # establecen el ancho mínimo de los contornos
min_contour_height = 40     # establecen el alto mínimo de los contornos
offset = 3                  # margen que se deja alrededor de la línea en la que se cuentan los coches que cruzan
line_height = 500           # altura de la línea en la que se cuentan los coches
matches = []                # lista que almacena las coordenadas 'x' e 'y' de los centros de los contornos detectados
cars = 0                    # recuento de coches detectados
 
 
def get_centrolid(x, y, w, h):
    # toma las coordenadas x, y, w y h de un contorno y devuelve las coordenadas del centroide
    x1 = int(w / 2)
    y1 = int(h / 2)
 
    cx = x + x1
    cy = y + y1
    return cx, cy
 

cap = cv2.VideoCapture('trafico01.mp4')     # abre el video 


if cap.isOpened(): ret, frame1 = cap.read() # si se ha abierto el video, frame1 = primer frame
else: ret = False                           # si no; no entra al bucle
# leen 2 frames consecutivos del video
ret, frame1 = cap.read()
ret, frame2 = cap.read()
 
 
while ret:  # mientras haya frames disponibles para leer
    # calcula la diferencia absoluta entre los 2 frames
    dif = cv2.absdiff(frame1, frame2)                 
    # convierte la imagen a escala de grises
    grey = cv2.cvtColor(dif, cv2.COLOR_BGR2GRAY)      
    # suaviza la imagen mediante el uso de una máscara gaussiana
    blur = cv2.GaussianBlur(grey, (5, 5), 0)          
    # aplica un umbral a la imagen suavizada para convertirla en una imagen binaria
    ret, th = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)       
    # dilata la imagen binaria para expandir las regiones blancas de la imagen binaria (pixeles adyacentes también se vuelvan blancos)
    dilated = cv2.dilate(th, np.ones((3, 3)))                       
    # crea un kernel que se utilizará en las operaciones morfológicas subsiguientes. El kernel será una elipse con un tamaño de (2, 2)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))   
    # dilata y erosiona la imagen para cerrar pequeños agujeros dentro de los objetos
    closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel)    
    # detectar contornos en la imagen (Los contornos se devuelven como una lista de puntos)
    contours, h = cv2.findContours(closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
    # itera a través de la lista de contornos y asigna el índice y el contorno a las variables i y c
    for(i, c) in enumerate(contours):   
        # obtiene los valores de (x, y) para el punto más a la izquierda y más arriba del rectángulo que envuelve el contorno 
        # y los valores de w y h para el ancho y alto del rectángulo, respectivamente.
        (x, y, w, h) = cv2.boundingRect(c)  
        # un contorno es válido si su ancho y alto son mayores o iguales a los valores mínimos establecidos para el ancho y alto del contorno
        contour_valid = (w >= min_contour_width) and (h >= min_contour_height)  
        # si el contorno no es válido, se salta al siguiente contorno
        if not contour_valid: continue  
        # dibuja un rectángulo alrededor del contorno en la imagen
        cv2.rectangle(frame1, (x-10, y-10), (x+w+10, y+h+10), (255, 0, 0), 2)   
        # dibuja una línea en la imagen
        cv2.line(frame1, (215, line_height), (1185, line_height), (200, 255, 0), 2) 
        # obtiene el centro del contorno y lo almacena en la variable centrolid.
        centrolid = get_centrolid(x, y, w, h)   
        # agrega el centro del contorno a la lista matches
        matches.append(centrolid)               
        # dibuja un círculo en el centro del contorno en la imagen frame1.
        cv2.circle(frame1, centrolid, 1, (255, 0, 300), 3)
        # obtienen el centro del contorno y lo almacenan en las variables cx y cy   
        cx, cy = get_centrolid(x, y, w, h)      
        # itera a través de la lista matches y cuenta el número de contornos que cruzan la línea dibujada en la imagen
        for (x, y) in matches:                  
            # Si el eje 'y' del centro del contorno está dentro de un cierto margen de la línea (determinado por la variable offset):
            if y < (line_height+offset) and y > (line_height-offset): 
                # se aumenta el contador de coches  
                cars = cars+1           
                # se elimina el contorno de la lista matches
                matches.remove((x, y))
    
    # pone texto en la pantalla
    cv2.putText(frame1, "Total de coches detectados: " + str(cars), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 0, 0), 2)
 
    cv2.imshow("Vehicle Detection", frame1)     # muestra el video
    #cv2.imshow("Difference" , th)              # muestra e video con el fondo negro coches en blanco
    if cv2.waitKey(1) == 27: break              # acabar si se preciona la tecla esc    
    frame1 = frame2                             # el frame1 pasa a ser el frame2, ya que los comparamos arriba
    ret, frame2 = cap.read()                    # frame2 pasa a ser el siguiente frame y frame1 sera el anterior

cv2.destroyAllWindows()                         # elimina todas las pestañas
cap.release()                                   # cierra el archivo de video

In [11]:
import cv2

video = cv2.VideoCapture("trafico01.mp4")

# Crea un detector de objetos
detector = cv2.CascadeClassifier("cars.xml")

while True:
    # Si estás procesando un vídeo, obtén el siguiente fotograma
    success, image = video.read()
    if not success:
        break

    # Detecta los coches en la imagen o vídeo
    cars = detector.detectMultiScale(image)

    # Recorre todos los coches detectados
    for i, (x, y, w, h) in enumerate(cars):
        # Dibuja un rectángulo alrededor del coche
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
        # Escribe el ID del coche en el rectángulo
        cv2.putText(image, str(i), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Muestra los resultados en una ventana de visualización
    cv2.imshow("Imagen con coches detectados", image)
    cv2.waitKey(1)

# Cierra la ventana de visualización y libera los recursos utilizados por cv2
cv2.destroyAllWindows()
video.release()


SystemError: <class 'cv2.CascadeClassifier'> returned a result with an exception set